Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = ""
COLLABORATORS = ""

---

<a href="https://colab.research.google.com/github/janboone/datascience_course/blob/master/Statistical_Hacking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Why statistical hacking?

In order to get started on the "big data" part of this course, it is important that you have a good intuition about a number of statistical results that you have seen before in your statistics and econometrics courses. We go over these concepts by simulating our own data. This serves two goals:

* you fully understand where the data comes from (you generated it yourself)
* we train your programming skills in generating the data.

Being able to generate your data using python simulations is a first step in understanding how to deal with "real" data. 

Based on [this lecture](https://www.youtube.com/watch?v=Iq9DzN6mvYA) and [this one.](https://www.youtube.com/watch?v=VR52vSbHBAk)

If you like, there is also [free book](https://greenteapress.com/wp/think-stats-2e/) on this topic using python.

We are going to discuss the following topics. First, as you probably know, estimated parameters have a distribution. To illustrate, you may recall that some statistics have a t-distribution. When you move into machine learning, the models get so complicated that there are no analytical results on the distributions of estimated coefficients. So how do you get a sense of uncertainty in that case? You simulate the distribution. Below we show this with examples where you actually know (or should know) what the relevant distributions are.

Sometimes you only have a sample and no knowledge about the underlying model. Then you cannot simulate the distribution. But you can use the sample to learn something about the uncertainty underlying your parameters. This is called bootstrapping.

In Economics we are not only interested in predicting variables (like economic growth or stock prices) but we also want to influence them through policy interventions. For this we need to know causal links between variables; not just correlations. Although many people have the intuition that it is better to control for more variables in  a regression, this intuition is actually not correct. By controlling for some variables, you actually mess up your interpretation of an effect in a regression.

Finally, with "big data" it is very tempting to use a lot of variables in your models. You have got all these variables in your "big data set" so why not use them? This brings us to the concepts of over- and underfitting.

# Loading packages

To generate the data, we are going to use [tensorflow](https://www.tensorflow.org/). If you are running this in colab, install `tensorflow` and `linearmodels` by un-commenting the next cell.

Run the other cell to import all the packages that we need.

In [ ]:
#!pip install tensorflow==2.0.0-alpha0
#!pip install linearmodels

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
from scipy import optimize
import statsmodels.api as sm # check the error that cannot import name 'factorial' in from scipy.misc import factorial
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import tensorflow as tf
import altair as alt
from linearmodels.iv import IV2SLS
from tensorflow import keras


from tensorflow.keras import datasets, layers, models

#alt.renderers.enable('notebook')

# Distributions of an estimator

Let's start simple and consider a uniform distribution on $[0,1]$. We take a sample of size `sample_size` and calculate the mean $m$ of this sample. This variable $m$ is a sample statistic. We want to understand the properties of this sample statistic.

In most courses that you have done, you will have used statistical theory here. E.g. what is the distribution of $m$?

For simple models, like calculating the mean of a sample or doing an OLS regression, there is theory that describes what the distributions are of the mean or an estimated slope parameter. However, with big data techniques, you are going to use models where such theoretical results do not exist. So how do you figure out then what is happening?

This is where the hacking comes in. The simple idea is: we program the statistical problem and then run it "lots of times", say 10,000 times. This gives us a distribution.

## distribution of a sample average and sample standard deviation

For the example of $m$, we do this in the next code cell.

We draw `N_simulations` times a sample of `sample_size` and put this in a matrix (tensor) of size `N_simulations` by `sample_size`. Then we take the mean within a sample. Put differently, we take the mean over the second dimension ("columns") of the matrix. As python starts counting from 0, this means we take the mean over `axis=1`.

In [ ]:
N_simulations = 10000
sample_size = 10
simulated_data = np.mean(tf.random.uniform([N_simulations,sample_size],0,1),axis=1)

The code above calculated `N_simulations` averages (of samples of size `sample_size`). 

**Question** Plot a histogram of `simulated_data` using `matplotlib`.

**Question** When considering this distribution, which "theorem" is at work here?

**Exercise** Play around with `sample_size` to see what the effect is of different values for this variable.

**Exercise** Calculate the standard deviation of $m$; how does this depend on `sample_size`? Can you plot the standard deviation of $m$ against sample size? Which functional form is this?

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Above we considered the distribution of the sample mean $m$. But any parameter has a distribution; e.g. also the standard deviation of the sample $s$ has a distribution. Note that the standard deviation of the sample $s$ is **not** the same as the standard error $\sigma/\sqrt{n}$ which is the standard deviation of $m$'s distribution. Do not worry if you find the last sentence confusing in the beginning...

**Question** Plot the distribution of $s$. What is the probability that $s<0.28$?

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

## distribution of a slope

If we run a simple OLS (ordinary least squares) linear regression, we find the constant and the slope of this line. These parameters have also distributions! We can plot these distributions as well.

We first generate the data with which we want to estimate a regression line.

**Question** Explain the code below.

In [ ]:
N_simulations = 10000
sample_size = 20
slope = 0.5
constant = 1.0
noise = 0.1
simulated_x = tf.random.normal([N_simulations,sample_size])
simulated_y = constant + slope * simulated_x + noise*tf.random.normal([N_simulations,sample_size])

We create a vector `constants` and a vector `slopes`. For each of our simulations we add the resulting constant and slope to their resp. vectors.

Note that the method `.numpy()` turns the tensorflow tensors into numpy arrays which are easier to work with.

In [ ]:
constants = []
slopes = []

for i in range(N_simulations):
    model = sm.OLS(simulated_y[i,:].numpy(), sm.add_constant(simulated_x[i,:].numpy()))
    results = model.fit().params
    constants.append(results[0])
    slopes.append(results[1])

**Question** Plot the distribution of slopes and the distribution of constants.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Instead of plotting the slopes and constants separately, we can plot the lines that are induced by these distributions of the constant and the slope.

**Exercise** Plot in $(x,y)$ space, the distribution of lines that follow from these distributions of slopes and constants.

**Question** For which values of $x$ is the uncertainty about $y$ the biggest?

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

# Bootstrapping

Sometimes we do not know what the underlying model is that generated the data. We only have the sample that we observed.  How do we proceed in this case if we want to test properties of the data? Here we can use bootstrapping.



Suppose that we have two populations (e.g. men vs women; or in the test of a new drug: patients that got the treatment and patients that received the placebo). Denote these two samples by $A$ and $B$. 



In [ ]:
sample_size = 50
delta = 0.95
A = 10+tf.random.normal([sample_size])
B = A*delta
plt.hist(A,label='sample A')
plt.hist(B,alpha=0.3,label='sample B')
plt.legend()

**Question** Show that the average of $A$ is higher than the average of $B$.


In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

But is this difference in means a significant effect (forget for a second that you saw the code generating the data)?

In your statistics class you have seen a [t-test](https://en.wikipedia.org/wiki/Student%27s_t-test) to figure out whether this difference is significant or not. But we are going to use simulations to establish this.

The null hypothesis that we want to test is: samples $A$ and $B$ are drawn from the same distribution.

If this is true, than we can simply combine the $A$ and $B$ observations, reshuffle them and calculate the difference. We do this 10,000 times and get a distribution of the difference.

In [ ]:
AB = tf.concat([A,B],axis=0).numpy()

In [ ]:
differences = []

for i in range(10000):
  np.random.shuffle(AB)
  differences.append(np.mean(AB[:sample_size])-np.mean(AB[sample_size:]))
                             
                      

In [ ]:
plt.hist(differences)
plt.show()

In [ ]:
sum(differences>observed_difference)/len(differences)

# Doing your own OLS

There exist many packages that can do OLS for you. But it is illustrative, to program your own OLS estimator, because then you can see what it does.

We start really simple and through the lecture we are going to make the estimator more sophisticated.

For OLS, we could just substitute the solution using matrix algebra: $\beta = (x^Tx)^{-1}x^Ty$. But we want to use the optimization problem itself so that we can add things to it later on.

First, we are going to generate our own data.

Then we define a loss function (mean squared error) and minimize the loss by choosing a constant (`w[0]`) and slope (`w[1]`).

In [ ]:
sample_size = 500
slope = -3.0
constant = 1.0
x = tf.random.normal([sample_size])
y = slope*x+constant+3*tf.random.normal([sample_size])

def loss(w):
    return np.sum((w[0]+w[1]*x-y)**2)/len(y)

optimize.fmin(loss, [0,0])

## What are ridge and lasso regressions?




In [ ]:
def loss_ridge(w,λ):
    return np.sum((w[0]+w[1]*x-y)**2)/len(y)+λ*(w[0]**2+w[1]**2)

optimize.fmin(lambda w: loss_ridge(w,1), [0,0])

In [ ]:
def loss_lasso(w,λ):
    return np.sum((w[0]+w[1]*x-y)**2)/len(y)+λ*(np.abs(w[0])+np.abs(w[1]))

optimize.fmin(lambda w: loss_lasso(w,1), [0,0])

# Causality

The data we generate is inspired by [Richard McElreath
's lecture 6](https://www.youtube.com/watch?v=l_7yIUqWBmE). 

As you know, multiple regression is about correlations, not about causality. When looking at the results of a multiple regression, there are a number of mistakes that you can make. Here we are going to look at four of these mistakes:

* the fork
* the pipe
* the collider

## The fork

This is the situation where there is no relation between the variables $X$ and $Y$. However, both are influenced by a third variable $Z$. 



![fork](./Fork.png)


Doing a regression from $X$ on $Y$ seems to suggest a causal effect of $X$ on $Y$. This could misleadingly convince people that a policy that affects $X$ directly will also affect $Y$.

However, using both $X$ and $Z$ as explanatory variables in the regression will show that there is no significant effect from $X$ on $Y$.

Hence, with a fork it is important to think about possible variables $Z$ and include them in your regression.

**Question** Generate data for the variables $X,Y,Z$ such that there is a fork

**Question** Put this data in a pandas dataframe.

With the dataframe, it is easy to run the regressions.

## The pipe

![pipe](./Pipe.png)

## The collider

Many people have the intuition that adding a control variable to a regression (especially when it turns out to be signficant) is a good idea. However, this intuition is wrong if it is applied without thinking.

We use here the education example by Richard McElreath to illustrate this point. Consider the educational achievements of three generations in one family: the grandparent, the parent and the (grand)child. We are interested in the effect of the grandparent's education on the educational achievement of the grandchild. 

Let's denote the child's educational achievement by $Y$ (the variable we are interested in) and the grandparent's education $X$. We want to understand the effect of $x$ on $Y$. Clearly there is the following path: more educated grandparent leads to more educated parent which, in turn, leads to more educated grandchild. The thing we are interested in is whether there is a direct effect from the grandparent on the child, *controlling* for the parent's education.

![collider](./Collider.png)

We know that the parent's education has a positive effect on the child's achievement. A more educated parent will tend to stimulate their children more, can help with homework etc. In the data that we generate below, we assume that $Z = X + ...$ and $Y = Z + ...$. That is, we assume that the parent's education feeds one-to-one into the child's education. 

Is it the case that on top of this effect there is a separate grandparent effect? E.g. because the grandparent is babysitting and a more educated grandparent reads Hamlet with the 5 year old grandchild. That is fun and can boost the child's educational achievement.

Another important component of education is the neighborhood where the child grows up. We denote this variable $U$. In the code we assume that the parent and child grow up in the same neighborhood (have the same $U$ effect).  $U$ is drawn from a standard normal distribution and the effect on education is given through a multiplication by the factor `alpha`.

In addition to this, we assume that there is a random component as well which differs between parent and child.

![collider](./Collider2.png)

The code below, generates this data and a dataframe `df` that stores this data.

Look carefully at the code: is there a direct effect from the grandparent to the grandchild's educational achievement?

We generate two dataframes: `df` is the one that is observed and used in the analysis by the researcher. 

We will use `df_2` to illustrate why one can get incorrect results from analyzing `df`. The problem is that the researcher does not have access to `df_2` but we can analyze `df_2` for "educational purposes" to illustrate why problems arise.

In [ ]:
N_observations = 5000
alpha = 0.5
X = tf.random.normal([N_observations])
U = tf.random.normal([N_observations])
Z = X + alpha*U+0.4*tf.random.normal([N_observations])
Y = Z + alpha*U+0.4*tf.random.normal([N_observations])

df = pd.DataFrame({'X':X, 'Y':Y, 'Z':Z})
df_2 = pd.DataFrame({'X':X, 'Y':Y, 'Z':Z, 'U':U})
print(df.head())
print(df_2.head())

To find the (direct) effect of the grandparent ($X$) and the grandchild's educational achievement ($Y$) conditional on the parent ($Z$), we use multiple regression where both $x$ and $Z$ are used as explanatory variables:

In [ ]:
results = smf.ols('Y ~ X + Z', data=df).fit()
print(results.summary())

The effect of the grandparent's educational achievement ($X$) on the child's achievement is negative and significant. Where does that come from? How is this even possible?

If we consider the direct effect of $X$ on $Y$ it is --as expected-- positive and significant:

In [ ]:
results2 = smf.ols('Y ~ X', data=df).fit()
print(results2.summary())


To see what is happening here in a graph, let's think about what it means to "control for parental education $Z$". 

To visualize what happens, we condition on $Z$ by focusing on a narrow band of $Z$ observations. In other words, we condition on $Z$ by focusing on a subsample where the $Z$ values are (almost) the same. We do this by creating a column `selected` in our dataframe.

In [ ]:
df['selected'] = np.abs(df.Z)<0.05

In [ ]:
df.head()

Feel free to make this plot using `matplotlib`. For fun, we use `altair` here. 

The blue circles are all the data, the orange ones are the data with (almost) the same $Z$.

**Question** Give the intuition why there is a negative correlation between $X$ and $Y$ with the orange dots. [hint: what do you know about orange dots with low $X$ and about orange dots with high $X$?]

If you cannot answer this question yet, consider the following interactive graph.



In [ ]:

chart = alt.Chart(df).mark_point().encode(
  x='X',
  y='Y',
  color='selected'
).interactive()

chart.save('Chart.html')

In [ ]:
%%HTML

<iframe width="840" height="400" src="./Chart.html" frameborder="0"></iframe>

To better understand what is happening here, we are going to use `df_2`. In particular, in the combined figure below, we again provide a scatter plot of `X` vs `Y`. Now the color indicates the level of `Z` associated with the observation; darker colors indicate higher values of `Z`. The size of the point (circle) indicates the level of `U` (that the researcher cannot see in `df`). 

To condition on `Z` (as we do in a multiple regression), we can drag a rectangle in the bottom histogram. Move this histogram around and see what happens. In particular:
* how does `Z` vary in the figure? E.g. where are the high values of `Z`? What is the interpretation of this?
* for a selection of `Z` in the bottom histogram, what is the relation between `X` and `Y`?
* for this selection, how does `U` vary in the figure? What is the interpretation?


**Question** Give the intuition why there is a negative correlation between `X` and `Y` for a selection of `Z`; that is, conditional on `Z`.

**Exercise** What is the risk of saying: I have run a regression and controlled for all effects that I had variables for?

In [ ]:
interval = alt.selection_interval(encodings=['y'])

fig = alt.Chart(df_2).mark_point().encode(
  x='X',
  y='Y',
  color=alt.condition(interval, 'Z', alt.value('lightgreen')),
  size = 'U'
).properties(
    selection=interval

)



hist = alt.Chart(df_2).mark_bar().encode(
    x='count()',
    y=alt.Y('Z',bin=True),
    color=alt.condition(interval, 'Z', alt.value('lightgrey'))
).properties(
    selection=interval

)

chart2 = fig & hist

chart2.save('Chart2.html')

In [ ]:
%%HTML

<iframe width="840" height="800" src="./Chart2.html" frameborder="0"></iframe>

# Some background on tensors

We will have a technical interlude to explain what tensors are and what you can do with them. We describe some functions that can be applied to tensors. To motivate these functions, we will build our first neural network. The point here is not so much to explain how neural networks works but more to show you that the functions we consider are going to be useful later on when we will try to understand neural networks more deeply.

## Tensors

In most of the empirical analyses that you have done, an observation is a one dimensional vector. E.g. you have data on inflation, unemployment, gdp growth for country-year combinations. Then for the UK in 2000, our data  consists of the one dimensional vector `[inflation, unemployment, gdp growth]`. If you have 100 observations like this, you can represent them in a matrix. Each row is an observation and the columns will be `[coutry, year, inflation, unemployment, gdp growth]`. Hence we have a two dimensional dataset with 100 rows and 5 columns.

In big data, there are usually higher dimension observations. One of the "classic" datasets in machine learning is [the MNIST dataset.](https://en.wikipedia.org/wiki/MNIST_database) This dataset consists of handwritten numbers and their corresponding label (e.g. when the handwritten number is 5, the label for this image `5`). We will see an example shortly.

The point is that a handwritten number is a two dimensional observation. To illustrate this, consider the 5th image from the training dataset (python starts numbering at 0); we plot the handwritten image (which is 2 dimensional) and print its label.



In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

In [ ]:
plt.imshow(train_images[4],cmap=plt.cm.binary)
plt.show()
print(train_labels[4])

Indeed, the handwritten number is 9, which equals its label.

What are the dimensions for this dataset. Let's use `shape` to find out:

In [ ]:
train_images.shape

In [ ]:
train_images[0].shape

## Tensors in numpy

Well known machine learning backends are Theano and Tensorflow. For reasons that we do not worry about here, these are not immediately straightforward to work with. But we can play around with dimensions in Numpy as well. Properties that we can use in Numpy, like broadcasting, can be used in Theano and Tensorflow as well.

As Numpy is "more direct" to work with, we will practice this with numpy.

First, we create the vector `x` with 100 random numbers. 

In [ ]:
x = np.random.normal(0,1,size=100)

We can check what the shape is of this vector:

In [ ]:
x.shape

and the vector is one-dimensional from a tensor point of view:

In [ ]:
x.ndim

**No not read this if you get easily confused:** If you would plot the vector `x` it would be a *vector* in a 100-dimensional space. Hence it is a 100 dimensional vector, but a 1 dimensional tensor.


Let us know consider a two dimensional (2D) tensor. This is what we usually call a matrix

In [ ]:
x2 = x.reshape(25,4)

In [ ]:
x2.ndim

In [ ]:
x2.shape

In matrix terminology we would say that `x2` has 25 rows and 4 columns.

A 3D tensor is then

In [ ]:
x3 = x.reshape(4,5,5)

In [ ]:
x3.ndim

In [ ]:
x3.shape

One way to think about this is as four $5*5$ images.

The numpy representation of this is as follows:

In [ ]:
x3

we can also add a dimension to a vector without adding or re-arranging data. For this we use `np.newaxis`

In [ ]:
x4 = x3[:,:,:,np.newaxis]

In [ ]:
x4.ndim

In [ ]:
x4.shape

Now you may wonder why this is a useful thing to do.

## Broadcasting

Suppose you have estimated a fixed effect model. In your data there are i = 0,1,...,9 individuals and t = 0,1,2 periods. The vector `I` consists of 10 individual fixed effects and the vector `T` has 3 time fixed effects. Hence, our prediction for indiv. `i` in time period `t` (ignoring other explanatory variables) is $y_{it}=I_i+T_t$. 

How can we calculate the vector `y`?

We first give you the answer using broadcasting and then we step back to explain what broadcasting is.




In [ ]:
I = np.arange(10)
T = np.arange(0,30,10)

In [ ]:
I

In [ ]:
T

Because of the simplistic numbers that we have chosen, it is easy to see that e.g. $y_{31} = 13$ with $i=3$ and $t=1$. So we can check whether our trick works.

In [ ]:
y = I[:,np.newaxis]+T[np.newaxis,:]

In [ ]:
y

In [ ]:
I[:,np.newaxis]

In [ ]:
T[np.newaxis,:]

What is going on here?

## Slicing and reshape

## First neural network

In [ ]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])


model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])



In [ ]:
model.fit(train_images, train_labels, epochs=5)


In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

print('\nTest accuracy:', test_acc)


In [ ]:
predictions = model.predict(test_images)
predictions[0]

In [ ]:
np.argmax(predictions[0])


In [ ]:
test_labels[0]


In [ ]:
plt.imshow(test_images[0],cmap=plt.cm.binary)
plt.show()

## Functions

What is `relu` and `softmax`? What does `adam` mean?


In [ ]:
np.maximum(z,0)

In [ ]:
np.relu(10,0)

In [ ]:
np.dot

# Overfitting and underfitting


Adding more explanatory variables to a regression cannot make the fit worse. This is easy to see: if it would become worse, you would set the coefficient on the added variables to 0. That would give the same fit as before. In real applications that may not be completely true as there is often randomness in the routines that are being used.

underfitting is ...

overfitting is ...


So how can we choose what the right model is?

A simple idea is cross validation. This idea is seen a lot in data science. Here we consider a simple example to illustrate cross validation.






## Regression in tensorflow

Below we will run regressions for a number of models. To understand what happens, let us first run a simple regression to see what the syntax looks like.

There are a number of python packages with which you can do a regression analysis. For example, [statsmodels](https://www.statsmodels.org/stable/index.html) has a nice syntax and by default returns things like t-values etc. If you just need to run a regression or two, than this is a great choice.

But if we want to program a series of regressions, tensorflow or [scikit-learn](https://scikit-learn.org/stable/index.html) are more useful as they are easier to program.

Here we use tensorflow.

First, we generate some simple data and add these to a dataframe.






In [ ]:
N_observations = 50
train_size = 25
x = tf.random.normal([N_observations])
y = 3*x+5*tf.random.normal([N_observations])
df = pd.DataFrame({'y':y,'x':x})
df['constant'] = 1


In [ ]:
features = ['constant','x']

y_train = df['y'][:train_size]
y_test =  df['y'][train_size:]

In [ ]:
def make_input_fn(data_df, label_df, num_epochs=10, batch_size=32):
  def input_function():
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
    ds = ds.batch(batch_size).repeat(num_epochs)
    return ds
  return input_function

In [ ]:
df_train = df[features][:train_size]
df_test = df[features][train_size:]
NUMERIC_COLUMNS = features
feature_columns = []
for feature_name in NUMERIC_COLUMNS:
    feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

train_input_fn = make_input_fn(df_train, y_train)
eval_input_fn = make_input_fn(df_test, y_test)
  
linear_est = tf.estimator.LinearRegressor(feature_columns=feature_columns)
linear_est.train(train_input_fn)

In [ ]:
linear_est.evaluate(train_input_fn)


## Series of regressions


To illustrate overfitting, we construct a dataset, generating $y$ values from some $x$ vector

In [ ]:
N_observations = 50
train_size = 25
x = tf.random.normal([N_observations])
y = 100*x+np.cos(x)-4*np.exp(x)+x**2+x**5-x**3+55*tf.random.normal([N_observations])
df = pd.DataFrame({'y':y,'x':x})
df['constant'] = 1
df['x2'] = df.x**2
df['x3'] = df.x**3
df['x4'] = df.x**4
df['x5'] = df.x**5
df['x6'] = df.x**6
df['x7'] = df.x**7
df['x8'] = df.x**8
df['x9'] = df.x**9
df['x10'] = df.x**10


In [ ]:
features = ['constant','x','x2','x3','x4','x5','x6','x7','x8','x9','x10']

y_train = df['y'][:train_size]
y_test =  df['y'][train_size:]

In [ ]:
def make_input_fn(data_df, label_df, num_epochs=10, shuffle=False, batch_size=32):
    def input_function():
        ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
        if shuffle:
            ds = ds.shuffle(1000)
        ds = ds.batch(batch_size).repeat(num_epochs)
        return ds
    return input_function


In [ ]:
def make_input_fn(data_df, label_df, num_epochs=10, batch_size=32):
    def input_function():
        ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
        ds = ds.batch(batch_size).repeat(num_epochs)
        return ds
    return input_function

train_score = []
test_score = []
number_features = []

for i in range(2,len(features)+1):
    df_train = df[features[:i]][:train_size]
    df_test = df[features[:i]][train_size:]
    NUMERIC_COLUMNS = features[:i]
    feature_columns = []
    for feature_name in NUMERIC_COLUMNS:
        feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))
      
    train_input_fn = make_input_fn(df_train, y_train)
    eval_input_fn = make_input_fn(df_test, y_test)
  
    linear_est = tf.estimator.LinearRegressor(feature_columns=feature_columns)
    linear_est.train(train_input_fn)
  
    print(i)
    number_features.append(i)
    train_score.append(linear_est.evaluate(train_input_fn)['average_loss'])
    test_score.append(linear_est.evaluate(eval_input_fn)['average_loss'])
  
   


In [ ]:
outcome = pd.DataFrame({'x_i':number_features, 'train_score': train_score, 'test_score': test_score})  
outcome

The following graph shows that adding more terms $x^a$ always helps to get a better fit on the training data. The loss measure falls as we add more terms.

In [ ]:
plt.plot(range(2,12),outcome.train_score)


But this is not the case with the test data. We fitted the model on the training data. Now we consider how well the model predicts "out-of-sample" on the test data. Adding 6 terms helps to predict on the test data. However, with more terms added to the model, the model starts to pick up features that are specific to our sample in the training data. These features are nor general to the underlying process. Hence, our loss increases on the test data.

By adding more than 6 terms, we start to overfit in this example.

In [ ]:
plt.plot(range(2,12),np.log(outcome.test_score))


# OLS using Tensorflow

Using more functionality from Tensorflow to program our OLS.

This allows us to see some building blocks that we will need below to program our neural network.

# Cross validation


k-fold cross-validation

# Neural network

To get a first sense of how a neural network works, we look at two simple python programs that implement these algorithms. For this, we use the website from the book [machine learning: an algorithmic perspective](http://homepages.ecs.vuw.ac.nz/~marslast/MLbook.html).

Below you find the code for the files [pcn_logic_eg.py](http://homepages.ecs.vuw.ac.nz/~marslast/Code/Ch3/pcn_logic_eg.py) and [mlp.py](http://homepages.ecs.vuw.ac.nz/~marslast/Code/Ch4/mlp.py); which we adapted for python 3 by adjusting the print statements (should be `print()` in python 3).

By looking at the underlying code we get an idea how neural networks work. Then we create a neural network using tensorflow 2.0. This part is based on a [tensorflow 2.0 tutorial](https://github.com/tensorflow/docs/blob/master/site/en/r2/tutorials/estimators/premade_estimators.ipynb).

The tensorflow syntax is very similar to [Keras](https://keras.io/). There is a [datacamp course](https://www.datacamp.com/courses/deep-learning-in-python) on deep learning with Keras.

##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

First we install/import the libraries that we need.

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import pandas as pd
import numpy as np

In [ ]:
from scipy import optimize
import matplotlib.pyplot as plt
import altair as alt

## Perceptron

Next we copy from Chapter 3 of Machine Learning: An Algorithmic Perspective, the "perceptron". The code is in the next cell. Try to understand the python.




The main part of the algorithm is in the function (method, actually, but do not worry about this) `pcntrain`; the line

`self.weights -= eta*np.dot(np.transpose(inputs),self.activations-targets)`

determines how the weights in the network are updated. This optimization step is discussed in [this chapter of the datacamp course.](https://campus.datacamp.com/courses/deep-learning-in-python/optimizing-a-neural-network-with-backward-propagation?ex=1)



In [ ]:
# Code from Chapter 3 of Machine Learning: An Algorithmic Perspective (2nd Edition)
# by Stephen Marsland (http://stephenmonika.net)

# You are free to use, change, or redistribute the code in any way you wish for
# non-commercial purposes, but please maintain the name of the original author.
# This code comes with no warranty of any kind.

# Stephen Marsland, 2008, 2014



class pcn:
	""" A basic Perceptron (the same pcn.py except with the weights printed
	and it does not reorder the inputs)"""
	
	def __init__(self,inputs,targets):
		""" Constructor """
		# Set up network size
		if np.ndim(inputs)>1:
			self.nIn = np.shape(inputs)[1]
		else: 
			self.nIn = 1
	
		if np.ndim(targets)>1:
			self.nOut = np.shape(targets)[1]
		else:
			self.nOut = 1

		self.nData = np.shape(inputs)[0]
	
		# Initialise network
		self.weights = np.random.rand(self.nIn+1,self.nOut)*0.1-0.05

	def pcntrain(self,inputs,targets,eta,nIterations):
		""" Train the thing """	
		# Add the inputs that match the bias node
		inputs = np.concatenate((inputs,-np.ones((self.nData,1))),axis=1)
	
		# Training
		change = range(self.nData)

		for n in range(nIterations):
			
			self.activations = self.pcnfwd(inputs);
			self.weights -= eta*np.dot(np.transpose(inputs),self.activations-targets)
			print("Iteration: ", n)
			print(self.weights)
			
			activations = self.pcnfwd(inputs)
			print("Final outputs are:")
			print(activations)
		#return self.weights

	def pcnfwd(self,inputs):
		""" Run the network forward """

		# Compute activations
		activations =  np.dot(inputs,self.weights)

		# Threshold the activations
		return np.where(activations>0,1,0)

	def confmat(self,inputs,targets):
		"""Confusion matrix"""

		# Add the inputs that match the bias node
		inputs = np.concatenate((inputs,-np.ones((self.nData,1))),axis=1)
		outputs = np.dot(inputs,self.weights)
	
		nClasses = np.shape(targets)[1]

		if nClasses==1:
			nClasses = 2
			outputs = np.where(outputs>0,1,0)
		else:
			# 1-of-N encoding
			outputs = np.argmax(outputs,1)
			targets = np.argmax(targets,1)

		cm = np.zeros((nClasses,nClasses))
		for i in range(nClasses):
			for j in range(nClasses):
				cm[i,j] = np.sum(np.where(outputs==i,1,0)*np.where(targets==j,1,0))

		print(cm)
		print(np.trace(cm)/np.sum(cm))




Below you see 4 data points with a target value (0 or 1) for each data point. We need to predict the target value. We plot this with a red color for target 0 and blue for target 1.

Predicting the target here means finding a straight line such that all red points are on one side of the line and the blue points on the other side of the line. Since this example is very simple, you can draw the line yourself (I hope...). This simplicity helps us to understand what the algorithm does.

We define a tensor with 4 `inputs` and 4 `targets`. The first element of `inputs` (which has two coordinates) corresponds to the first element in `targets`.

To refer to the points in the figure, we use "standard" terminology $x$ and $y$.


In [ ]:
inputs = np.array([[0,0],[0,1],[1,0],[1,1]])
targets = np.array([[0],[1],[1],[1]])
colors = []
for i in range(len(targets)):
    colors.append(['red','blue'][targets[i][0]])
plt.scatter(inputs[:,0],inputs[:,1],color=colors)
plt.xlabel('$x$')
plt.ylabel('$y$')
plt.show()

Now we use the `pcn` class and instantiate it with the `inputs` and `targets` above. We call `p` and instance of the class `pcn`. In this course we will not dwell on the object oriented aspects of python, so do not worry if you do not fully understand this. However, as you will see below, when we use tensorflow, we have a similar structure: we create an instance of a tensorflow class.

The function definition is `pcntrain(self,inputs,targets,eta,nIterations)`. Do not worry about the `self` part. We call the function as `pcntrain(inputs,targets,0.25,6)`. Hence, the first two arguments of the function are our `inputs` and `targets` tensors. 

**Continue**

The result is that `p` is now associated with our tensors `inputs` and `targets`. Now we can use the function ("method") `pcntrain` to train the network. After each iteration, the algorithm prints some information. Looking at `pcntrain` above, we can see that the information that is printed is:

```
print("Iteration: ", n)
print(self.weights)
print("Final outputs are:")
print(activations)
```

So first we see the number of the iteration (and python starts counting at 0). Then we see three numbers that correspond to the weights. The weights determine the line in the figure above. In particular, if we write the weights tensor as $w = (w_0,w_1,w_2)$, then a line in the figure is given by $w_0 x + w_1 y =w_2$. Equivalently, we can write this as:

\begin{equation}
y = (w_2-w_0 x)/w_1
\end{equation}

Next, we see the final outputs (targets): this is the "prediction" of the algorithm for the vector `targets`. Recall that `targets` is of the form [0,1,1,1]. Hence, after the first iteration, we incorrectly label the first point as 1 while it should be 0. Hence, the algorithm continues and generates different (and ultimately better) predictions.


In [ ]:
p = pcn(inputs,targets)
p.pcntrain(inputs,targets,0.25,6)

The final weights $w$ are now equal to:

In [ ]:
p.weights

We can now plot the line $y=(w_2-w_0 x)/w_1$. Indeed, as you can see, the line sepates the red point (below the line) from the blue points (above the line). If we give the algorithm a point above the line, it will predict "blue", if we give it a point below the line it will predict "red". 

Because we only have 4 points here, this is arbitrary for points close to the line: we can shift the line, still separate the points, but get different predictions for points close to the line.

In [ ]:
x0 = 0
x1 = 1
y0 = (p.weights[2]-p.weights[0]*x0)/p.weights[1]
y1 = (p.weights[2]-p.weights[0]*x1)/p.weights[1]
plt.scatter(inputs[:,0],inputs[:,1],color=colors)
plt.plot([x0,x1],[y0,y1],'k') #we only need to give two points to plot a line
plt.xlabel('$x$')
plt.xlabel('$y$')
plt.show()

We can see from the graph and check with the function `pcnfwd` below what the predicted labels will be for the points (0.5,0.5) and (0.5,-0.5). The `-1` is needed for the "constant" $w_2$.

In [ ]:
p.pcnfwd([[0.5,0.5,-1],[0.5,-0.5,-1]])

## Multi-layer perceptron

With more than one layer, we get into deep learning.

We use [this mlp code](http://homepages.ecs.vuw.ac.nz/~marslast/Code/Ch4/mlp.py) from Chapter 4 of Machine Learning: An Algorithmic Perspective.

In [ ]:

# Code from Chapter 4 of Machine Learning: An Algorithmic Perspective (2nd Edition)
# by Stephen Marsland (http://stephenmonika.net)

# You are free to use, change, or redistribute the code in any way you wish for
# non-commercial purposes, but please maintain the name of the original author.
# This code comes with no warranty of any kind.

# Stephen Marsland, 2008, 2014

import numpy as np

class mlp:
    """ A Multi-Layer Perceptron"""
    
    def __init__(self,inputs,targets,nhidden,beta=1,momentum=0.9,outtype='logistic'):
        """ Constructor """
        # Set up network size
        self.nin = np.shape(inputs)[1]
        self.nout = np.shape(targets)[1]
        self.ndata = np.shape(inputs)[0]
        self.nhidden = nhidden

        self.beta = beta
        self.momentum = momentum
        self.outtype = outtype
    
        # Initialise network
        self.weights1 = (np.random.rand(self.nin+1,self.nhidden)-0.5)*2/np.sqrt(self.nin)
        self.weights2 = (np.random.rand(self.nhidden+1,self.nout)-0.5)*2/np.sqrt(self.nhidden)

    def earlystopping(self,inputs,targets,valid,validtargets,eta,niterations=100):
    
        valid = np.concatenate((valid,-np.ones((np.shape(valid)[0],1))),axis=1)
        
        old_val_error1 = 100002
        old_val_error2 = 100001
        new_val_error = 100000
        
        count = 0
        while (((old_val_error1 - new_val_error) > 0.001) or ((old_val_error2 - old_val_error1)>0.001)):
            count+=1
            print(count)
            self.mlptrain(inputs,targets,eta,niterations)
            old_val_error2 = old_val_error1
            old_val_error1 = new_val_error
            validout = self.mlpfwd(valid)
            new_val_error = 0.5*np.sum((validtargets-validout)**2)
            
        print("Stopped", new_val_error,old_val_error1, old_val_error2)
        return new_val_error
    	
    def mlptrain(self,inputs,targets,eta,niterations):
        """ Train the thing """    
        # Add the inputs that match the bias node
        inputs = np.concatenate((inputs,-np.ones((self.ndata,1))),axis=1)
        change = range(self.ndata)
    
        updatew1 = np.zeros((np.shape(self.weights1)))
        updatew2 = np.zeros((np.shape(self.weights2)))
            
        for n in range(niterations):
    
            self.outputs = self.mlpfwd(inputs)

            error = 0.5*np.sum((self.outputs-targets)**2)
            if (np.mod(n,100)==0):
                print("Iteration: ",n, " Error: ",error)

            # Different types of output neurons
            if self.outtype == 'linear':
            	deltao = (self.outputs-targets)/self.ndata
            elif self.outtype == 'logistic':
            	deltao = self.beta*(self.outputs-targets)*self.outputs*(1.0-self.outputs)
            elif self.outtype == 'softmax':
                deltao = (self.outputs-targets)*(self.outputs*(-self.outputs)+self.outputs)/self.ndata 
            else:
            	print("error")
            
            deltah = self.hidden*self.beta*(1.0-self.hidden)*(np.dot(deltao,np.transpose(self.weights2)))
                      
            updatew1 = eta*(np.dot(np.transpose(inputs),deltah[:,:-1])) + self.momentum*updatew1
            updatew2 = eta*(np.dot(np.transpose(self.hidden),deltao)) + self.momentum*updatew2
            self.weights1 -= updatew1
            self.weights2 -= updatew2
                
            # Randomise order of inputs (not necessary for matrix-based calculation)
            #np.random.shuffle(change)
            #inputs = inputs[change,:]
            #targets = targets[change,:]
            
    def mlpfwd(self,inputs):
        """ Run the network forward """

        self.hidden = np.dot(inputs,self.weights1);
        self.hidden = 1.0/(1.0+np.exp(-self.beta*self.hidden))
        self.hidden = np.concatenate((self.hidden,-np.ones((np.shape(inputs)[0],1))),axis=1)

        outputs = np.dot(self.hidden,self.weights2);

        # Different types of output neurons
        if self.outtype == 'linear':
        	return outputs
        elif self.outtype == 'logistic':
            return 1.0/(1.0+np.exp(-self.beta*outputs))
        elif self.outtype == 'softmax':
            normalisers = np.sum(np.exp(outputs),axis=1)*np.ones((1,np.shape(outputs)[0]))
            return np.transpose(np.transpose(np.exp(outputs))/normalisers)
        else:
            print("error")

    def confmat(self,inputs,targets):
        """Confusion matrix"""

        # Add the inputs that match the bias node
        inputs = np.concatenate((inputs,-np.ones((np.shape(inputs)[0],1))),axis=1)
        outputs = self.mlpfwd(inputs)
        
        nclasses = np.shape(targets)[1]

        if nclasses==1:
            nclasses = 2
            outputs = np.where(outputs>0.5,1,0)
        else:
            # 1-of-N encoding
            outputs = np.argmax(outputs,1)
            targets = np.argmax(targets,1)

        cm = np.zeros((nclasses,nclasses))
        for i in range(nclasses):
            for j in range(nclasses):
                cm[i,j] = np.sum(np.where(outputs==i,1,0)*np.where(targets==j,1,0))

        print("Confusion matrix is:")
        print(cm)
        print("Percentage Correct: ",np.trace(cm)/np.sum(cm)*100)

This code is more elaborate than the perceptron above. It also introduces new concepts like the confusion matrix. We will go over these below. 

Before we use tensorflow with pandas dataframes, we first use more primitive datastructures in `numpy`. This serves two purposes: first, it is simple and helps to understand the underlying python code and algorithms; second, sometimes the data that you get is too messy to fit into a dataframe right away. Then it is useful to know some more primitive structures as well.

Below we read the famous [iris dataset](http://archive.ics.uci.edu/ml/datasets/Iris). Admittedly, flowers are not directly linked to economics, but this is one of the classic datasets in classification. You cannot claim to have done "datascience" without having looked at the iris dataset.

We will first download it from the website, just to learn the relevant steps here. We will work with in numpy and then we turn it into a pandas dataframe. In fact, the data is so famous that it is included in the tensorflow library, but later you may want to analyze data that is not included in any library. Hence, we need to learn the download steps as well.

When you look at the downloaded data (in an editor), you will notice that it actually includes the names as strings. For us it is easier to work with numerical labels 0,1,2. Hence, we use the function `preprocessIris`. This function takes two arguments: the input file with the data that we downloaded and the output file where the target-labels are replaced by 0,1,2.

If you run this notebook for the first time, un-comment the line `preprocessIris('iris.data','iris_proc.data')` (that is, remove the hashtag "#" at the beginning of the line), where `iris.data` is the name of the file that you downloaded and `iris_proc.data` is the name of the file with labels 0,1,2. Once the file `iris_proc.data` is created, there is no need to run this python command again.

**continue here**

In [ ]:
# Code from Chapter 4 of Machine Learning: An Algorithmic Perspective (2nd Edition)
# by Stephen Marsland (http://stephenmonika.net)

# You are free to use, change, or redistribute the code in any way you wish for
# non-commercial purposes, but please maintain the name of the original author.
# This code comes with no warranty of any kind.

# Stephen Marsland, 2008, 2014

# The iris classification example

def preprocessIris(infile,outfile):

    stext1 = 'Iris-setosa'
    stext2 = 'Iris-versicolor'
    stext3 = 'Iris-virginica'
    rtext1 = '0'
    rtext2 = '1'
    rtext3 = '2'

    fid = open(infile,"r")
    oid = open(outfile,"w")

    for s in fid:
        if s.find(stext1)>-1:
            oid.write(s.replace(stext1, rtext1))
        elif s.find(stext2)>-1:
            oid.write(s.replace(stext2, rtext2))
        elif s.find(stext3)>-1:
            oid.write(s.replace(stext3, rtext3))
    fid.close()
    oid.close()

## Preprocessor to remove the test (only needed once)
# preprocessIris('iris.data','iris_proc.data')

iris = np.loadtxt('./data/iris_proc.data',delimiter=',')
iris[:,:4] = iris[:,:4]-iris[:,:4].mean(axis=0)
imax = np.concatenate((iris.max(axis=0)*np.ones((1,5)),np.abs(iris.min(axis=0)*np.ones((1,5)))),axis=0).max(axis=0)
iris[:,:4] = iris[:,:4]/imax[:4]
print(iris[0:5,:])

# Split into training, validation, and test sets
target = np.zeros((np.shape(iris)[0],3));
indices = np.where(iris[:,4]==0) 
target[indices,0] = 1
indices = np.where(iris[:,4]==1)
target[indices,1] = 1
indices = np.where(iris[:,4]==2)
target[indices,2] = 1

# Randomly order the data
order = np.arange(np.shape(iris)[0])
np.random.shuffle(order)
iris = iris[order,:]
target = target[order,:]

train = iris[::2,0:4]
traint = target[::2]
valid = iris[1::4,0:4]
validt = target[1::4]
test = iris[3::4,0:4]
testt = target[3::4]

#print train.max(axis=0), train.min(axis=0)

# Train the network
#import mlp
net = mlp(train,traint,5,outtype='logistic')
net.earlystopping(train,traint,valid,validt,0.1)
net.confmat(test,testt)


In [ ]:
preprocessIris('./data/iris.data','./data/iris_proc2.data')

In [ ]:
iris

# Premade Estimators

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/alpha/tutorials/estimators/premade_estimators"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/r2/tutorials/estimators/premade_estimators.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/tree/master/site/en/r2/tutorials/estimators/premade_estimators.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>


This tutorial shows you
how to solve the Iris classification problem in TensorFlow using Estimators. An Estimator is TensorFlow's high-level representation of a complete model, and it has been designed for easy scaling and asynchronous training. For more details see
[Estimators](https://www.tensorflow.org/guide/estimators).

Note that in TensorFlow 2.0, the [Keras API](https://www.tensorflow.org/guide/keras) can accomplish many of these same tasks, and is believed to be an easier API to learn. If you are starting fresh, we would recommend you start with Keras. For more information about the available high level APIs in TensorFlow 2.0, see [Standardizing on Keras](https://medium.com/tensorflow/standardizing-on-keras-guidance-on-high-level-apis-in-tensorflow-2-0-bad2b04c819a).



## First things first

In order to get started, you will first import TensorFlow and a number of libraries you will need.

## The data set

The sample program in this document builds and tests a model that
classifies Iris flowers into three different species based on the size of their
[sepals](https://en.wikipedia.org/wiki/Sepal) and
[petals](https://en.wikipedia.org/wiki/Petal).


You will train a model using the Iris data set. The Iris data set contains four features and one
[label](https://developers.google.com/machine-learning/glossary/#label).
The four features identify the following botanical characteristics of
individual Iris flowers:

* sepal length
* sepal width
* petal length
* petal width

Based on this information, you can define a few helpful constants for parsing the data:


In [ ]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

Next, download and parse the Iris data set using Keras and Pandas. Note that you keep distinct datasets for training and testing.

In [ ]:
train = pd.DataFrame({'SepalLength':iris[::2,0],'SepalWidth':iris[::2,1],'PetalLength':iris[::2,2],'PetalWidth':iris[::2,3],'Species':iris[::2,4]})
valid = pd.DataFrame({'SepalLength':iris[1::4,0],'SepalWidth':iris[1::4,1],'PetalLength':iris[1::4,2],'PetalWidth':iris[1::4,3],'Species':iris[1::4,4]})
test = pd.DataFrame({'SepalLength':iris[3::4,0],'SepalWidth':iris[3::4,1],'PetalLength':iris[3::4,2],'PetalWidth':iris[3::4,3],'Species':iris[3::4,4]})

In [ ]:
train['Species']=train.Species.astype(int)
valid['Species']=valid.Species.astype(int)
test['Species']=test.Species.astype(int)

In [ ]:
train.head()

In [ ]:
train_y = train.pop('Species')
valid_y = valid.pop('Species')
test_y = test.pop('Species')

# The label column has now been removed from the features.
train.head()

You can inspect your data to see that you have four float feature columns and one int32 label.

## Overview of programming with Estimators

Now that you have the data set up, you can define a model using a TensorFlow Estimator. An Estimator is any class derived from `tf.estimator.Estimator`. TensorFlow
provides a collection of
`tf.estimator`
(for example, `LinearRegressor`) to implement common ML algorithms. Beyond
those, you may write your own
[custom Estimators](https://www.tensorflow.org/guide/custom_estimators).
We recommend using pre-made Estimators when just getting started.

To write a TensorFlow program based on pre-made Estimators, you must perform the
following tasks:

* Create one or more input functions.
* Define the model's feature columns.
* Instantiate an Estimator, specifying the feature columns and various
  hyperparameters.
* Call one or more methods on the Estimator object, passing the appropriate
  input function as the source of the data.

Let's see how those tasks are implemented for Iris classification.

## Create input functions

You must create input functions to supply data for training,
evaluating, and prediction.

An **input function** is a function that returns a `tf.data.Dataset` object
which outputs the following two-element tuple:

* [`features`](https://developers.google.com/machine-learning/glossary/#feature) - A Python dictionary in which:
    * Each key is the name of a feature.
    * Each value is an array containing all of that feature's values.
* `label` - An array containing the values of the
  [label](https://developers.google.com/machine-learning/glossary/#label) for
  every example.

Just to demonstrate the format of the input function, here's a simple
implementation:

In [ ]:
def input_evaluation_set():
    features = {'SepalLength': np.array([6.4, 5.0]),
                'SepalWidth':  np.array([2.8, 2.3]),
                'PetalLength': np.array([5.6, 3.3]),
                'PetalWidth':  np.array([2.2, 1.0])}
    labels = np.array([2, 1])
    return features, labels

In [ ]:
input_evaluation_set()

Your input function may generate the `features` dictionary and `label` list any
way you like. However, we recommend using TensorFlow's [Dataset API](https://www.tensorflow.org/guide/datasets), which can
parse all sorts of data.

The Dataset API can handle a lot of common cases for you. For example,
using the Dataset API, you can easily read in records from a large collection
of files in parallel and join them into a single stream.

To keep things simple in this example you are going to load the data with
[pandas](https://pandas.pydata.org/), and build an input pipeline from this
in-memory data:


In [ ]:
def input_fn(features, labels, training=True, batch_size=256):
    """An input function for training or evaluating"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)


## Define the feature columns

A [**feature column**](https://developers.google.com/machine-learning/glossary/#feature_columns)
is an object describing how the model should use raw input data from the
features dictionary. When you build an Estimator model, you pass it a list of
feature columns that describes each of the features you want the model to use.
The `tf.feature_column` module provides many options for representing data
to the model.

For Iris, the 4 raw features are numeric values, so we'll build a list of
feature columns to tell the Estimator model to represent each of the four
features as 32-bit floating-point values. Therefore, the code to create the
feature column is:

In [ ]:
# Feature columns describe how to use the input.
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

Feature columns can be far more sophisticated than those we're showing here.  You can read more about Feature Columns in [this guide](https://www.tensorflow.org/guide/feature_columns).

Now that you have the description of how you want the model to represent the raw
features, you can build the estimator.

## Instantiate an estimator

The Iris problem is a classic classification problem. Fortunately, TensorFlow
provides several pre-made classifier Estimators, including:

* `tf.estimator.DNNClassifier` for deep models that perform multi-class
  classification.
* `tf.estimator.DNNLinearCombinedClassifier` for wide & deep models.
* `tf.estimator.LinearClassifier` for classifiers based on linear models.

For the Iris problem, `tf.estimator.DNNClassifier` seems like the best choice.
Here's how you instantiated this Estimator:

In [ ]:
# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 10 nodes each.
    hidden_units=[30, 10],
    # The model must choose between 3 classes.
    n_classes=3)

## Train, Evaluate, and Predict

Now that you have an Estimator object, you can call methods to do the following:

* Train the model.
* Evaluate the trained model.
* Use the trained model to make predictions.

### Train the model

Train the model by calling the Estimator's `train` method as follows:

In [ ]:
# Train the Model.
classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps=5000)

Note that you wrap up your `input_fn` call in a
[`lambda`](https://docs.python.org/3/tutorial/controlflow.html)
to capture the arguments while providing an input function that takes no
arguments, as expected by the Estimator. The `steps` argument tells the method
to stop training after a number of training steps.


### Evaluate the trained model

Now that the model has been trained, you can get some statistics on its
performance. The following code block evaluates the accuracy of the trained
model on the test data:


In [ ]:
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(test, test_y, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

on the validation set:

In [ ]:
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(valid, valid_y, training=False))

print('\nValidation set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

Unlike the call to the `train` method, you did not pass the `steps`
argument to evaluate. The `input_fn` for eval only yields a single
[epoch](https://developers.google.com/machine-learning/glossary/#epoch) of data.


The `eval_result` dictionary also contains the `average_loss` (mean loss per sample), the `loss` (mean loss per mini-batch) and the value of the estimator's `global_step` (the number of training iterations it underwent).


### Making predictions (inferring) from the trained model

You now have a trained model that produces good evaluation results.
You can now use the trained model to predict the species of an Iris flower
based on some unlabeled measurements. As with training and evaluation, you make
predictions using a single function call:

In [ ]:
# Generate predictions from the model
expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth': [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth': [0.5, 1.5, 2.1],
}

def input_fn(features, batch_size=256):
    """An input function for prediction."""
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

predictions = classifier.predict(
    input_fn=lambda: input_fn(valid))

The `predict` method returns a Python iterable, yielding a dictionary of
prediction results for each example. The following code prints a few
predictions and their probabilities:

In [ ]:
for pred_dict, expec in zip(valid, expected):
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%), expected "{}"'.format(
        SPECIES[class_id], 100 * probability, expec))

# Treatment effects

Machine learning is about predicting outcomes. When the prediction is accurate (out-of-sample) we are happy, almost no matter what the model looks like.

This is different when we are interested in policy advice. 

## IV

What are instrumental variables again?

We use the IV example in [Richard McElreath's lecture 18
](https://www.youtube.com/watch?v=e5cgiAGBKzI) (starting around 28:00) which is based on Angrist and Krueger (1991).

An individual's education (`e`) affects her wage (`w`). We want to know how strong this effect is.

`q` denotes fraction of the year that has elapsed when you were born.

**TODO** add text






In [ ]:
N_observations=200
alpha_w = 1.
beta_ew = 0.
alpha_e = 1.
beta_qe = 2.
q = tf.random.uniform([N_observations])
u = tf.random.normal([N_observations])
e = alpha_e + beta_qe*q + u + tf.random.normal([N_observations])
w = alpha_w + beta_ew*e + u + tf.random.normal([N_observations])
df = pd.DataFrame({'q':q,'e':e,'w':w})

In [ ]:
results = smf.ols('w ~ e', data=df).fit()
print(results.summary())

In [ ]:

mod = IV2SLS.from_formula('w ~ 1 + [e ~ q]', df)

iv_res = mod.fit()
iv_res

In [ ]:
iv_res.first_stage

## Heterogenous treatment effects

One of the complications when using IV to find causal effects is that treatment effects can differ between subgroups and that these subgroups react differently to the treatment/instrument. This is based on Chapter 4 of Angrist and Pischke (2009).

We will consider this in the context of unemployed who are offered a training program. To understand the effects, we will again generate our own data such that we know exactly what all the relevant effects are. The question is then, how do we get these effects from the data.

As before, the learning goals of this section are two fold: (i) learn how to program your own data generating process in python and (ii) understand the statistical issue under considerarion: heterogenous treatment effects.

The government introduces a training program for unemployed. The treatment variable $D_i$ denotes whether (1) or not (0) individual $i$ received training. The instrument $Z_i$ that we use is whether (1) or not (0) $i$ is invited to join the training. 

We have three groups of unemployed:

* compliers join the training if and only if they are invited to do so: $D_i = 1 (0)$ iff $Z_i = 1 (0)$;
* always-takers join the training whether or not they are invited to do so: $D_i =1$ irrespective of $Z_i$;
* never-takers never join the training: $D_i = 0$ irrespective of $Z_i$.

Note that in this training context, the always-takers are a bit "odd": how can you join a training program where you are not supposed to be. The never-takers make more sense in this context: some people may not attend the training even though they were invited to do so. We will not worry about this; the point is that we can conceptually distinguish these groups.

We are interested in the effect of the training program on earnings. We denote by $\beta$ the expected earnings without training. The expected earnings with training is denoted by $\beta+\tau$. Hence, $\tau$ denotes the treatment effects.

Let $j \in \{c,a,n \}$ denote whether an individual is a complier, always-taker or never-taker. We assume that $\beta_j$ and $\tau_j$ vary with $j$.

In particular we assume that the earnings effect for someone in group $j$ is drawn from a normal distribution with mean $\mu = \beta_j$ and standard deviation $\sigma =1$ in case of no training and the effect of training is drawn from a normal distribution with mean $\mu = \tau_j$ and standard deviation $\sigma =1$.

Below we will consider different configurations for $\beta_j, \tau_j$ and see how they affect the estimated effects.







In [ ]:
def accept_complier(invited):
    return invited

def accept_always(invited):
    return np.ones_like(invited)

def accept_never(invited):
    return np.zeros_like(invited)

N_simulations = 10000
fraction_training = 0.2 # 20% of agents are invited to the training
types = ['complier','always_taker', 'never_taker']
β = {'complier': 1, 'always_taker': 2, 'never_taker': 1}
τ = {'complier': 3, 'always_taker': 4, 'never_taker': 1}
n = {'complier': 0.5, 'always_taker': 0.25, 'never_taker': 0.25}
accept = {'complier': accept_complier, 'always_taker': accept_always, 'never_taker': accept_never}
earnings_without = {}
training_effect = {}
earnings = {}
invited = {}
trained = {}
df = pd.DataFrame()
def data_simulation(β = β, τ=τ, n = n):
    for j in types:
        earnings_without[j] = np.random.normal(loc = β[j], scale = 1.0, size = int(n[j]*N_simulations))
        training_effect[j] = np.random.normal(loc = τ[j], scale = 1.0, size = int(n[j]*N_simulations))
        invited[j] = np.random.binomial(1,fraction_training,size = int(n[j]*N_simulations))
        trained[j] = accept[j](invited[j])
        earnings[j] = earnings_without[j]+ trained[j]*training_effect[j]
    column_invited = np.concatenate([invited[j] for j in types],axis=0)
    column_trained = np.concatenate([trained[j] for j in types],axis=0)
    column_earnings = np.concatenate([earnings[j] for j in types],axis=0)
    df = pd.DataFrame({'invited':column_invited, 'trained':column_trained,'earnings':column_earnings})
    return df
 
df = data_simulation()
df.head()

We have a dataset now with indicators for who was invited to a training and who attended a training.

We would like to learn the effect of training on earnings.

**Question** Which of the parameters above could we hope to recover? $\tau_c,\tau_a,\tau_n$?

**Question** Hence, if we do some calculations with the data, which number do we want to see?

**Question** Compare the average earnings of people with training with the average earnings of people without training. Is this the number that you expected? Why (not)?

YOUR ANSWER HERE

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

**Exercise** Where does this number come from?


YOUR ANSWER HERE

**Question** Perhaps we should use the instrument whether someone if invited or not: compare average earnings of people with and without an invitation to the training.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

**Question** Does it help if we compare:
* people with training and an invitation to training with
* people without training and no invitation to training

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE

We have seen now that we have no hope of recovering the relevant training effect in the set-up above. So the question is: when can we recover the relevant parameters? That is, what assumptions are needed?

To see how you can analyze this with the set-up above, let's program two obvious cases where this works.

**Question** Explain for each case *why* it works.

In [ ]:
n = {'complier': 1.0, 'always_taker': 0.0, 'never_taker': 0.0}
df_compliers_only = data_simulation(n=n)
np.mean(df_compliers_only[df_compliers_only.trained==1].earnings)-np.mean(df_compliers_only[df_compliers_only.trained==0].earnings)

In [ ]:
β = {'complier': 1, 'always_taker': 1, 'never_taker': 1}
τ = {'complier': 3, 'always_taker': 3, 'never_taker': 3}
df_homog = data_simulation(β=β,τ=τ)
np.mean(df_homog[df_homog.trained==1].earnings)-np.mean(df_homog[df_homog.trained==0].earnings)

YOUR ANSWER HERE

For a less obvious case where we can retrieve relevant parameters, we will use Angrist and Pischke (2009: chapter 4). In particular, we will verify Theorem 4.4.2.

We work with one sided noncompliance. That is, we allow for never-takers but not for always-takers. 

According to Theorem 4.4.2 the effect of treatment on the treated can be found as:

$$
\frac{E(Y_i|Z_i=1)-E(Y_i|Z_i=0)}{Prob(D_i=1|Z_i=1)}
$$

We first create the dataframe.

In [ ]:
n = {'complier': 0.5, 'always_taker': 0.0, 'never_taker': 0.5}
df_one_sided = data_simulation(n=n)


**Question** Calculate $Prob(D_i = 1|Z_i =1)$ and denote this `correction_term`.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

**Question** Calculate $\frac{E(Y_i|Z_i=1)-E(Y_i|Z_i=0)}{Prob(D_i=1|Z_i=1)}$.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

**Question** Which parameter have we recovered?

YOUR ANSWER HERE

## Probability of treatment

Consider a situation which is simpler than the one above in the sense that there are no always-takers, not never-takers.

However, now the instrument ("invited") $Z$ causes the probability of training to increase from 0.2 to 0.6. 30% of people get invited.

**Exercise** Copy/paste python code from the analysis above to generate a dataframe using that $Z_i=1$ implies that the probability of treatment increases from 0.2 to 0.6

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

**Question** Suppose we can observe whether someone received training or not. Use this to calculate the effect of training on earnings.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Suppose we cannot observe whether someone actually did the training or not (e.g. whether or not someone invested effort in the training). We only know ("from a previous study") that explicitly inviting someone for the training ("nudge") increases the probability that they invest training effort from 0.2 to 0.6.

**Question** Use the dataframe above to verify what the effect of the nudge is. Call this the `correction_term`. What value should the correction term have?

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Angrist and Pischke (2009: chapter 4) Theorem 4.4.1 (LATE Theorem; local average treatment effect) claims that (under some assumptions), we can calculate the effect of training on earnings as:

$$
\frac{E(Y_i|Z_i=1)-E(Y_i|Z_i=0)}{E(D_i|Z_i=1)-E(D_i|Z_i=0)}
$$

**Question** Use this expression to calculate the effect of training on earnings.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Now we are going to use an IV estimation to find the same result. Run the following two regressions.

In [ ]:
results_first_stage = smf.ols('trained ~ invited', data=df_prob).fit()
results_second_stage = smf.ols('earnings ~ invited', data=df_prob).fit()

**Question** Use these regressions to calculate the effect of training on earning.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()